In [270]:
# coding : UTF-8
import requests
import csv
import random
import time
import socket
import http.client
import urllib.request
from bs4 import BeautifulSoup
import re
import json


In [121]:
def get_content(url, data = None):
    header = {
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
                'Accept-Encoding': 'gzip, deflate',
                'Accept-Language': 'zh-CN,zh;q=0.9',
                'Cache-Control': 'max-age=0',
                'Connection': 'keep-alive',
                'Cookie': 'blogAppAd_blog7article=1; UM_distinctid=16df8bd5d1c690-0051368edd1f57-38617b01-13c680-16df8bd5d1d35c; SINAGLOBAL=141.72.234.188_1573873288.794867; Apache=141.72.234.188_1573873288.794869; U_TRS1=000000bc.6e418963.5df18198.99133406; U_TRS2=000000bc.6e4c8963.5df18198.d4afaee3; SCF=AkmVSbL5FWmAU7oHvQuVc_VqMHdz4qnbxT3FQVC7GDmEUpEalQI984dPCVeLjVUagHBQbgWVWQpMABgBeEIurIQ.; ULOGIN_IMG=gz-10731209612149fe8422304673a37c6bb31a; SUB=_2AkMpV5ZWf8NxqwJRmP4UzWngbYp2yQHEieKfC2eNJRMyHRl-yD9jqkMitRB6Ate4uUHhYnHlXPyl7UMUvCsyOQImOF9N; SUBP=0033WrSXqPxfM72-Ws9jqgMF55529P9D9W5HTfE.DIjwwkk2J4PsMkwZ; UOR=,blog.sina.com.cn,; ULV=1579254411887:1:1:1:141.72.234.188_1573873288.794869:; BLOG_TITLE=%E9%98%86%E8%8B%91%E6%98%A5%E6%99%93%E7%9A%84%E5%8D%9A%E5%AE%A2',
                'Host': 'blog.sina.com.cn',
                'Upgrade-Insecure-Requests': '1',
                'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36',

    }
    timeout = random.choice(range(80, 180))
    while True:
        try:
            rep = requests.get(url, headers=header, timeout=timeout)
            rep.encoding = 'utf-8'
            req = urllib.request.Request(url, data, header)
            response = urllib.request.urlopen(req, timeout=timeout)
            html1 = response.read().decode('UTF-8', errors='ignore')
            response.close()
            break
        except urllib.request.HTTPError as e:
                print( '1:', e)
                time.sleep(random.choice(range(5, 10)))

        except urllib.request.URLError as e:
            print( '2:', e)
            time.sleep(random.choice(range(5, 10)))
        except socket.timeout as e:
            print( '3:', e)
            time.sleep(random.choice(range(8,15)))

        except socket.error as e:
            print( '4:', e)
            time.sleep(random.choice(range(20, 60)))

        except http.client.BadStatusLine as e:
            print( '5:', e)
            time.sleep(random.choice(range(30, 80)))

        except http.client.IncompleteRead as e:
            print( '6:', e)
            time.sleep(random.choice(range(5, 15)))

    return rep.text
    # return html_text

In [271]:
def get_data(html_text):
    final = []
    raw_poems = []
    title_author = []
    bs = BeautifulSoup(html_text,"html.parser")
    body = bs.body
    data = body.find('div',{'id':'sina_keyword_ad_area2'})
    p = data.find_all('p')#匹配所有带p的标签
    for i in p:
        raw_poem = i
        raw_poem_text = i.text.strip()
#         raw_poem = raw_poem.replace('\xa0', '')
        raw_poems.append(raw_poem_text)
    del raw_poems[0]
    processed_poems = list(filter(None,raw_poems))
#         processed_poems = raw_poems.pop(0)
    
#     print(p)
#     print(p[].string)
#     for i in p:
#         raw_poem = p[i]
       
#         raw_poems.append(raw_poem)  
#         final.append(raw_poems) 
    title_author = []
    poem_content = []
    poem_dic = {}
    d = 0
    for a in range(0,len(processed_poems),2):
        title_author.append(processed_poems[a][2:])
#         print(processed_poems[a][2:])
        caesura_pos = re.search("\、",processed_poems[a]).span()[1]
        left_pos = re.search("\（",processed_poems[a][caesura_pos:]).span()[0]#左括号位置
        
        title = processed_poems[a][caesura_pos:][:left_pos]#左括号以及之前的内容也就是题目
        
        raw_author = processed_poems[a][caesura_pos:][left_pos:]#包含括号的作者
        right_pos = re.search("\）",raw_author[1:]).span()[0]#右括号的位置
        
        author = raw_author[1:][:right_pos]

#       print(title)
#       print(author)
        poem_content.append(processed_poems[a+1])
#       print(title,poem_content)
        sub_dic = {}
        sub_dic["title"] = title
        sub_dic["author"] = author
        sub_dic["content"] = processed_poems[a+1]
        poem_dic[str(d)] = sub_dic
        d+=1
    poem_j = json.dumps(poem_dic, indent=4, ensure_ascii=False) 
    with open('collection.json','w') as wr:
        wr.write(poem_j)
    
    
    

In [272]:
url = 'http://blog.sina.com.cn/s/blog_4b064c140101jgc0.html'
html = get_content(url)
get_data(html)
